# 热门地爬取

In [11]:
import os
import time
import random
from fake_useragent import UserAgent  # 用于生成随机请求头
import requests
import pandas as pd

In [2]:
# 去哪儿热门景点excel文件保存路径
PLACE_EXCEL_PATH = 'C:/Users/123/Desktop/qunar_place.xlsx'

## 爬取景点列表

In [19]:
def spider_place(keyword, page):
    """
    爬取景点

    :param keyword: 搜索关键字
    :param page: 分页参数
    :return:
    """
    url = f'http://piao.qunar.com/ticket/list.json?keyword={keyword}&region=&from=mpl_search_suggest&page={page}'
    headers = {
        'Accept': 'application / json, text / javascript, * / *; q = 0.01',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Host': 'piao.qunar.com',
        'Referer': 'http://piao.qunar.com/ticket/list.htm?keyword=%E5%9B%BD%E5%BA%86%E6%97%85%E6%B8%B8%E6%99%AF%E7%82%B9&region=&from=mpl_search_suggest',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }
    # 没有用代理
    # 站大爷：http://ip.zdaye.com/dayProxy.html
    proxies = {'http': '118.24.172.149:1080',
               'https': '60.205.202.3:3128'}
    response = requests.get(url, headers={'User-Agent': UserAgent().random
                                          })
    # 提取景点信息
    place_list = get_place_info(response.json())
    print(place_list)
        
    # 保存景点信息
    save_excel(place_list)

## 解析json列表

In [18]:
def get_place_info(response_json):
    """
    解析json，获取想要字段
    :param response_json:
    :return:
    """
    sight_list = response_json['data']['sightList']
    place_list = []
    for sight in sight_list:
        try:
            goods = {'id': sight['sightId'],  # 景点id
                     'name': sight['sightName'],  # 景点名称
                     # 景点星级，使用get方法防止触发KeyError
                     'star': sight.get('star', '无'),
                     'score': sight.get('score', 0),  # 评分
                     'price': sight.get('qunarPrice', 0),  # 门票价格
                     'sale': sight.get('saleCount', 0),  # 销量
                     'districts': sight['districts'],  # 省 市 县
                     'point': sight['point'],  # 坐标
                     'intro': sight.get('intro', ''),  # 简介
                     }
            place_list.append(goods)
        except IndexError:
            print('error')
    return place_list

## 生成Excel文件

In [21]:
def save_excel(place_list):
    """
    将json数据生成excel文件
    :param place_list: 景点数据
    :return:
    """
    # pandas没有对excel没有追加模式，只能先读后写
    try:
        if os.path.exists(PLACE_EXCEL_PATH):
            df = pd.read_excel(PLACE_EXCEL_PATH)
            df = df.append(place_list)
        else:
            df = pd.DataFrame(place_list)
    except IndexError:
        print('error')
        
    writer = pd.ExcelWriter(PLACE_EXCEL_PATH)
    # columns参数用于指定生成的excel中列的顺序
    df.to_excel(excel_writer=writer,
                columns=['id', 'name', 'star', 'score', 'price', 'sale', 'districts', 'point', 'intro'], index=False,
                encoding='utf-8', sheet_name='去哪儿热门景点')
    writer.save()
    writer.close()

## 批量爬取

In [25]:
def patch_spider_place(keyword):
    """
    批量爬取淘去哪儿景点
    :param keyword: 搜索关键字
    :return:
    """
    # 写入数据前先清空之前的数据
    if os.path.exists(PLACE_EXCEL_PATH):
        os.remove(PLACE_EXCEL_PATH)
    # 批量爬取
    for i in range(1, 36):
        print(f'正在爬取 {keyword} 第{i}页')
        spider_place(keyword, i)
        # 设置一个时间间隔
        time.sleep(random.randint(2, 5))
    print('爬取完成！')
    

## 似乎是因为疫情关系，第二页开始就无法爬取了

In [22]:
patch_spider_place('国庆旅游景点')

正在爬取 国庆旅游景点 第1页
[{'id': 4152824504, 'name': '崇州国庆金秋田园灯会', 'star': '无', 'score': '0.0', 'price': 0, 'sale': 0, 'districts': '四川·成都·崇州市', 'point': '103.714815,30.712283', 'intro': '夜晚去田园花海，领略梦幻光影'}, {'id': 4166750013, 'name': '国庆跑马场', 'star': '无', 'score': '0.0', 'price': 0, 'sale': 0, 'districts': '河北·邯郸·磁县', 'point': '114.43083,36.514686', 'intro': '培训，娱乐为一体的大型跑马场，这里地势平坦，广袤辽阔，环境怡人，空气清新。是久居都市的人们放松心情的好去处。也是邯郸独具'}, {'id': 1617365751, 'name': '牛顿小镇中秋国庆灯展', 'star': '无', 'score': '0.0', 'price': 0, 'sale': 0, 'districts': '湖北·黄冈·湖北红安牛顿小镇', 'point': '114.623286,31.288769', 'intro': '牛顿小镇投资打造的本届红安县独具风格的灯展，融合赏灯景、品美食、游戏互动多种项目。'}]
正在爬取 国庆旅游景点 第2页
[]
error
正在爬取 国庆旅游景点 第3页
[]
error
正在爬取 国庆旅游景点 第4页
[]
error


KeyboardInterrupt: 

# 现成数据分析

In [23]:
import re

import numpy as np
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar

In [24]:
# 去哪儿热门景点excel文件保存路径
PLACE_EXCEL_PATH = 'C:/Users/123/Desktop/qunar_place.xlsx'
# 读取数据
DF = pd.read_excel(PLACE_EXCEL_PATH)
# 百度热力图模板
HOT_MAP_TEMPLATE_PATH = 'C:/Users/123/Desktop/hot_map_template.html'
# 生成的国庆旅游景点热力图
PLACE_HOT_MAP_PATH = 'C:/Users/123/Desktop/place_hot_map.html'

## 分析销量

In [26]:
def analysis_sale():
    """
    分析销量
    :return: 
    """
    # 引入全局数据
    global DF
    df = DF.copy()
    # 1、生成一个名称和销量的透视表
    place_sale = df.pivot_table(index='name', values='sale')
    # 2、根据销量排序
    place_sale.sort_values('sale', inplace=True, ascending=True)
    print(place_sale)
    # 3、生成柱状图
    place_sale_bar = (
        Bar()
            .add_xaxis(place_sale.index.tolist()[-20:])
            .add_yaxis("", list(map(int, np.ravel(place_sale.values)))[-20:])
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(
            title_opts=opts.TitleOpts(title="国庆旅游热门景点门票销量TOP20"),
            yaxis_opts=opts.AxisOpts(name="景点名称"),
            xaxis_opts=opts.AxisOpts(name="销量")
        )
    )
    place_sale_bar.render('place-sale-bar.html')

## 分析销售额

In [27]:
def analysis_amount():
    """
    分析销售额
    :return:
    """
    # 引入全局数据
    global DF
    df = DF.copy()
    amount_list = []
    for index, row in df.iterrows():
        try:
            # 销售额
            amount = row['price'] * row['sale']
        except Exception:
            amount = 0
        amount_list.append(amount)
    df['amount'] = amount_list
    # 生成一个名称和销量的透视表
    place_amount = df.pivot_table(index='name', values='amount')
    # 根据销售额排序
    place_amount.sort_values('amount', inplace=True, ascending=True)
    print(place_amount)
    # 生成柱状图
    place_amount_bar = (
        Bar()
            .add_xaxis(place_amount.index.tolist()[-20:])
            .add_yaxis("", list(map(int, np.ravel(place_amount.values)))[-20:])
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(
            title_opts=opts.TitleOpts(title="国庆旅游热门景点门票销售额TOP20"),
            yaxis_opts=opts.AxisOpts(name="景点名称"),
            xaxis_opts=opts.AxisOpts(name="销售额")
        )
    )
    place_amount_bar.render('place-amount-bar.html')

## 分析省份旅游景点数（待完成）

In [28]:
def analysis_province():
    """
    此功能未完成
    分析省份旅游景点数
    :return:
    """
    # 引入全局数据
    global DF
    df = DF.copy()
    province_list = []
    for item in df.districts:
        province_match = re.search(r'^([\u4e00-\u9fa5])*', item)
        province_list.append(province_match[0])
    print(province_list)
    df['province'] = province_list
    # province_star = df.groupby(['province', 'star'])
    province_star = df.groupby(['province', 'star']).agg({'star': 'count'})
    province_star_dict = {}
    star = ['3A', '4A', '5A', '无']
    star_index = 0
    prev = ''
    for province_star_index in province_star.index:
        province_index = province_star_index[1]
        if prev != province_index:
            star_index = 0
            province_star_dict['province_index'] = []
        while True:
            if province_index == star[star_index]:
                province_star.star[province_star_index]
                star_index += 1
        province_star_dict
    print(province_star)

## 调用百度地图api生成热力图

In [30]:
def analysis_point_sale():
    """
    生成热力图，使用百度地图api
    :return:
    """
    # 引入全局数据
    global DF
    df = DF.copy()
    point_sale_list = []
    for index, row in df.iterrows():
        # 构建坐标数据
        lng, lat = row['point'].split(',')
        count = row['sale']
        point_sale = {'lng': float(lng), 'lat': float(lat), 'count': count}
        point_sale_list.append(point_sale)
    print(point_sale_list)
    data = f'var points ={str(point_sale_list)};'
    
    ### 替换模板中的坐标数据
    with open(HOT_MAP_TEMPLATE_PATH, 'r',
              encoding="utf-8") as f1, open(PLACE_HOT_MAP_PATH,
                                            'w',
                                            encoding="utf-8") as f2:
        s = f1.read()
        # 替换数据
        s2 = s.replace('%data%', data)
        f2.write(s2)
        f1.close()
        f2.close()

## 推荐排行榜

In [31]:
def analysis_recommend():
    """
    瞎推荐排行榜，高评分、销量少、价格便宜
    :return:
    """
    # 引入全局数据
    global DF
    df = DF.copy()
    recommend_list = []
    for index, row in df.iterrows():
        try:
            # 瞎推荐系数算法
            recommend = (row['score'] * 1000) / (row['price'] * row['sale'])
        except ZeroDivisionError:
            recommend = 0
        recommend_list.append(recommend)
    df['recommend'] = recommend_list
    # 生成一个名称和瞎推荐系数的透视表
    place_recommend = df.pivot_table(index='name', values='recommend')
    # 根据瞎推荐系数排序
    place_recommend.sort_values('recommend', inplace=True, ascending=True)
    print(place_recommend)
    # 生成柱状图
    place_recommend_bar = (
        Bar()
            .add_xaxis(place_recommend.index.tolist()[-20:])
            .add_yaxis("", list(map(int, np.ravel(place_recommend.values)))[-20:])
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(
            title_opts=opts.TitleOpts(title="国庆旅游热门景点瞎推荐TOP20"),
            yaxis_opts=opts.AxisOpts(name="景点名称"),
            xaxis_opts=opts.AxisOpts(name="瞎推荐系数")
        )
    )
    place_recommend_bar.render('place-recommend-bar.html')


In [32]:
if __name__ == '__main__':
    analysis_sale()
    analysis_amount()
    # analysis_province()
    analysis_point_sale()
    analysis_recommend()

                 sale
name                 
崇州国庆金秋田园灯会        0.0
国庆跑马场             0.0
伦敦圣保罗大教堂          2.0
绕阳湖水利风景区          2.0
金阁寺               2.0
...               ...
秦始皇陵博物院（兵马俑）  10800.0
八达岭长城         11989.0
故宫            22433.0
横店影视城         27344.0
上海迪士尼乐园       47606.0

[527 rows x 1 columns]
                  amount
name                    
国庆跑马场                0.0
崇州国庆金秋田园灯会           0.0
龙庆峡                  8.0
金阁寺                  9.6
清水寺                 20.0
...                  ...
故宫              785155.0
秦始皇陵博物院（兵马俑）   1166400.0
珠海长隆海洋王国       1863938.0
横店影视城          2707056.0
上海迪士尼乐园       18756764.0

[527 rows x 1 columns]
[{'lng': 121.667917, 'lat': 31.149712, 'count': 47606}, {'lng': 113.544272, 'lat': 22.104641, 'count': 5149}, {'lng': 115.984178, 'lat': 40.446359, 'count': 6127}, {'lng': 121.915647, 'lat': 30.917713, 'count': 5017}, {'lng': 116.404015, 'lat': 39.912729, 'count': 4073}, {'lng': 110.485702, 'lat': 29.122834, 'count': 5353}, {'lng': 118.704

                recommend
name                     
Vlasoff Cay沙岛    0.000000
武夷宫              0.000000
横店梦幻谷            0.000000
横店明清宫苑           0.000000
梦巴马大型实景演出        0.000000
...                   ...
金鞭溪            110.000000
二条城            125.000000
清水寺            250.000000
龙庆峡            437.500000
金阁寺            520.833333

[527 rows x 1 columns]
